In [ ]:
!pip install torch torchvision

In [ ]:
!pip install scikit-learn

In [ ]:
!pip install tqdm

In [ ]:
!pip install pandas

In [1]:
import torch
import torch.nn as nn
from torchvision import models
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.transforms.functional import to_pil_image
from PIL import Image
import os
import pandas as pd
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
class MultiLabelResNet50(nn.Module):
    def __init__(self, num_classes, hidden_size = 512, dropout_rate=0.5):
        super(MultiLabelResNet50, self).__init__()
        
        # Load pre-trained ResNet50
        self.base_model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
        
        # Modify the fully connected layer for multi-label classification
        self.base_model.fc = nn.Sequential(
            nn.Linear(self.base_model.fc.in_features, hidden_size),  # New intermediate layer. ##512 --> 256
            nn.ReLU(),
            nn.Dropout(dropout_rate),  # Dropout to prevent overfitting ##0.5 --> 0.6
            nn.Linear(hidden_size, num_classes),  # Output layer
            nn.Sigmoid()  # Sigmoid for multi-label classification (soften the data)
            #nn.Tanh()  #This is between -1 and 1

           # nn.Linear(self.base_model.fc.in_features, num_classes),
           # nn.Sigmoid()  # Sigmoid activation for multi-label classification
        )

    def forward(self, x):
        return self.base_model(x)

In [4]:
class MultiLabelResNet50_2(nn.Module):
    def __init__(self, num_classes):
        super(MultiLabelResNet50_2, self).__init__()
        
        # Load pre-trained ResNet50
        self.base_model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
        
        # Modify the fully connected layer for multi-label classification
        self.base_model.fc = nn.Sequential(
            nn.Linear(self.base_model.fc.in_features, 256),  # New intermediate layer. ##512 --> 256
            nn.ReLU(),
            nn.Dropout(0.6),  # Dropout to prevent overfitting ##0.5 --> 0.6
            nn.Linear(256, num_classes),  # Output layer
            nn.Sigmoid()  # Sigmoid for multi-label classification (soften the data)
            #nn.Tanh()  #This is between -1 and 1

           # nn.Linear(self.base_model.fc.in_features, num_classes),
           # nn.Sigmoid()  # Sigmoid activation for multi-label classification
        )

    def forward(self, x):
        return self.base_model(x)

In [5]:
class NoFindingRefiner(nn.Module):
    def __init__(self, input_dim=6):  # 1 from image model + 5 pathology scores
        super(NoFindingRefiner, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            #nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.5), #was 0.5
            nn.Linear(64, 1),
            nn.Sigmoid()  # Because your scores range from 0 to 1
        )

    def forward(self, x):
        return self.model(x).squeeze(1)


In [6]:
class NoFindingRefiner2(nn.Module):
    def __init__(self, input_dim=6):  # 1 from image model + 5 pathology scores
        super(NoFindingRefiner2, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            #nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.5),  #0.3 for 3
            nn.Linear(128, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(0.3),  #0.3 for 3
            nn.Linear(32, 1),
            nn.Sigmoid()  # Because your scores range from 0 to 1
        )

    def forward(self, x):
        return self.model(x).squeeze(1)


In [7]:
class MultiLabelDenseNet121(nn.Module):
    def __init__(self, num_classes, hidden_dim = 512, dropout_rate=0.5):
        super(MultiLabelDenseNet121, self).__init__()

        # Load pre-trained DenseNet-121
        self.base_model = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)
        
        # Replace the classifier with a custom head
        self.base_model.classifier = nn.Sequential(
            nn.Linear(self.base_model.classifier.in_features, hidden_dim),  
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, num_classes),
            nn.Tanh()  # Output values in [-1, 1] for each class
        )

    def forward(self, x):
        return self.base_model(x)

## Run Model

In [10]:
model_pel = MultiLabelResNet50(num_classes=1).to(device)
model_pel.load_state_dict(torch.load('models/pe_lat_model.pth'))
model_pel.eval()

model_pef = MultiLabelResNet50(num_classes=1).to(device)
model_pef.load_state_dict(torch.load('models/pe_front_model.pth'))
model_pef.eval()

model_pet = MultiLabelResNet50(num_classes=1).to(device)
model_pet.load_state_dict(torch.load('models/best_pe1_model.pth'))
model_pet.eval()

model_cml = MultiLabelResNet50_2(num_classes=1).to(device)
model_cml.load_state_dict(torch.load('models/cm_lat_model2.pth'))
model_cml.eval()

model_cmf = MultiLabelResNet50(num_classes=1).to(device)
model_cmf.load_state_dict(torch.load('models/cm_front_model.pth'))
model_cmf.eval()

model_cmt = MultiLabelResNet50(num_classes=1).to(device)
model_cmt.load_state_dict(torch.load('models/best_cm0_model.pth'))
model_cmt.eval()

model_lo = MultiLabelResNet50(num_classes=1, hidden_size=256).to(device)
model_lo.load_state_dict(torch.load('amb_models/lo_partial/epoch_3.pth'))
model_lo.eval()

model_lo_fr = MultiLabelResNet50(num_classes=1, hidden_size=256).to(device)
model_lo_fr.load_state_dict(torch.load('amb_models/lo_partial_frontal/epoch_6.pth'))
model_lo_fr.eval()

model_lo_l = MultiLabelResNet50(num_classes=1, hidden_size=256).to(device)
model_lo_l.load_state_dict(torch.load('amb_models/lo_partial_lateral/epoch_4.pth'))
model_lo_l.eval()

model_lo_fr1 = MultiLabelResNet50(num_classes=1, hidden_size=256).to(device)
model_lo_fr1.load_state_dict(torch.load('amb_models/lo_partial_frontal_2/epoch_7.pth'))
model_lo_fr1.eval()

model_lo_l1 = MultiLabelResNet50(num_classes=1, hidden_size=256).to(device)
model_lo_l1.load_state_dict(torch.load('amb_models/lo_partial_lateral_2/epoch_8.pth'))
model_lo_l1.eval()

model_frl = MultiLabelResNet50_2(num_classes=1).to(device)
model_frl.load_state_dict(torch.load('models/best_fr4_model.pth'))
model_frl.eval()

model_frf = MultiLabelResNet50_2(num_classes=1).to(device)
model_frf.load_state_dict(torch.load('models/fr_front_model2.pth'))
model_frf.eval()

model_frt = MultiLabelResNet50_2(num_classes=1).to(device)
model_frt.load_state_dict(torch.load('models/fr_lat_model5.pth'))
model_frt.eval()

model_nf = MultiLabelResNet50(num_classes=1, hidden_size=256).to(device)
model_nf.load_state_dict(torch.load('amb_models/nf_partial/epoch_3.pth'))
model_nf.eval()

model_nf_fr = MultiLabelResNet50(num_classes=1, hidden_size=256).to(device)
model_nf_fr.load_state_dict(torch.load('amb_models/nf_partial_frontal/epoch_7.pth'))
model_nf_fr.eval()

model_nf_l = MultiLabelResNet50(num_classes=1, hidden_size=256).to(device)
model_nf_l.load_state_dict(torch.load('amb_models/nf_partial_lateral/epoch_3.pth'))
model_nf_l.eval()

model_ecl = MultiLabelResNet50(num_classes=1, hidden_size = 128).to(device)
model_ecl.load_state_dict(torch.load('models/ec_lat_model.pth'))
model_ecl.eval()

model_ecf = MultiLabelResNet50(num_classes=1, hidden_size = 256).to(device)
model_ecf.load_state_dict(torch.load('models/ec_front_model.pth'))
model_ecf.eval()

model_ect = MultiLabelResNet50(num_classes=1).to(device)
model_ect.load_state_dict(torch.load('models/best_ec1_model.pth'))
model_ect.eval()

model_sd = MultiLabelResNet50(num_classes=1, hidden_size = 256).to(device)
model_sd.load_state_dict(torch.load('amb_models/sd_partial/epoch_2.pth'))
model_sd.eval()

'''model_sd_fr = MultiLabelResNet50(num_classes=1, hidden_size = 256).to(device)
model_sd_fr.load_state_dict(torch.load('amb_models/sd_partial_frontal/epoch_5.pth'))
model_sd_fr.eval()

model_sd_l = MultiLabelResNet50(num_classes=1, hidden_size = 256).to(device)
model_sd_l.load_state_dict(torch.load('amb_models/sd_partial_lateral/epoch_5.pth'))
model_sd_l.eval()

model_sd3 = MultiLabelResNet50(num_classes=1, hidden_size = 256).to(device)
model_sd3.load_state_dict(torch.load('amb_models/sd_partial_2/epoch_5.pth'))
model_sd3.eval()'''

model_pn = MultiLabelResNet50(num_classes=1, hidden_size = 256).to(device)
model_pn.load_state_dict(torch.load('amb_models/pn_partial/epoch_6.pth'))
model_pn.eval()

'''model_pn_fr = MultiLabelResNet50(num_classes=1, hidden_size = 512).to(device)
model_pn_fr.load_state_dict(torch.load('amb_models/pn_partial_frontal/epoch_9.pth'))
model_pn_fr.eval()

model_pn_l = MultiLabelResNet50(num_classes=1, hidden_size = 512).to(device)
model_pn_l.load_state_dict(torch.load('amb_models/pn_partial_lateral/epoch_7.pth'))
model_pn_l.eval()'''

"model_pn_fr = MultiLabelResNet50(num_classes=1, hidden_size = 512).to(device)\nmodel_pn_fr.load_state_dict(torch.load('amb_models/pn_partial_frontal/epoch_9.pth'))\nmodel_pn_fr.eval()\n\nmodel_pn_l = MultiLabelResNet50(num_classes=1, hidden_size = 512).to(device)\nmodel_pn_l.load_state_dict(torch.load('amb_models/pn_partial_lateral/epoch_7.pth'))\nmodel_pn_l.eval()"

In [11]:
import torch
import os
import pandas as pd
from tqdm import tqdm

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Directories
dir_dict = {
    "frontal": "input_images/solution_frontal",  ##NOTE THIS CHANGED TO SOLUTION IMAGES
    "lateral": "input_images/solution_lateral"
}

# Column setup
columns = ["Id", "No Finding", "Enlarged Cardiomediastinum", "Cardiomegaly", "Lung Opacity", 
           "Pneumonia", "Pleural Effusion", "Pleural Other", "Fracture", "Support Devices"]

average_values = {
    "No Finding": -0.734655,
    "Enlarged Cardiomediastinum": -0.275805,
    "Cardiomegaly": 0.190770,
    "Lung Opacity": 0.836288,
    "Pneumonia": 0.031183,
    "Pleural Other": 0.521795,
    "Fracture": 0.392374,
    "Support Devices": 0.888289
}

batch_size = 64
predictions = []

# Loop over both frontal and lateral directories
for view_type, test_dir in dir_dict.items():
    model_pe = model_pef if view_type == "frontal" else model_pel
    model_cm = model_cmf if view_type == "frontal" else model_cml
    model_ec = model_ecf if view_type == "frontal" else model_ecl
    model_fr = model_frf if view_type == "frontal" else model_frl
    model_lo2 = model_lo_fr if view_type == "frontal" else model_lo_l
    model_lo3 = model_lo_fr1 if view_type == "frontal" else model_lo_l1
    model_nf2 = model_nf_fr if view_type == "frontal" else model_nf_l
    #model_sd2 = model_sd_fr if view_type == "frontal" else model_sd_l
    #model_pn2 = model_pn_fr if view_type == "frontal" else model_pn_l
    file_list = [f for f in os.listdir(test_dir) if f.endswith(".pt")]
    
    batch = []
    batch_filenames = []

    for filename in tqdm(file_list, desc=f"Processing {view_type}"):
        image_path = os.path.join(test_dir, filename)
        image_tensor = torch.load(image_path).to(device)
        batch.append(image_tensor)
        batch_filenames.append(filename.split('.')[0])

        if len(batch) == batch_size or filename == file_list[-1]:
            input_batch = torch.stack(batch)

            with torch.no_grad():
                output_pe1 = model_pe(input_batch).cpu().numpy()
                output_cm1 = model_cm(input_batch).cpu().numpy()
                output_fr1 = model_fr(input_batch).cpu().numpy()
                output_ec1 = model_ec(input_batch).cpu().numpy()

                output_pe2 = model_pet(input_batch).cpu().numpy()
                output_cm2 = model_cmt(input_batch).cpu().numpy()
                output_fr2 = model_frt(input_batch).cpu().numpy()
                output_ec2 = model_ect(input_batch).cpu().numpy()

                output_pe = (output_pe1 + output_pe2) / 2
                output_cm = (output_cm1 + output_cm2) / 2
                output_fr = (output_fr1 + output_fr2) / 2
                output_ec = (output_ec1 + output_ec2) / 2

                output_lo = model_lo(input_batch).cpu().numpy()
                output_lo2 = model_lo2(input_batch).cpu().numpy()
                output_lo3 = model_lo3(input_batch).cpu().numpy()

                output_nf = model_nf(input_batch).cpu().numpy()
                output_nf2 = model_nf2(input_batch).cpu().numpy()

                output_sd = model_sd(input_batch).cpu().numpy()
                #output_sd2 = model_sd2(input_batch).cpu().numpy()
                #output_sd3 = model_sd3(input_batch).cpu().numpy()

                output_pn = model_pn(input_batch).cpu().numpy()
                #output_pn2 = model_pn(input_batch).cpu().numpy()


            for i in range(len(batch)):
                #output_pn[i][0] = (output_pn[i][0] + output_pn2[i][0])/2
                #output_sd[i][0] = (output_sd[i][0] + output_sd2[i][0] + output_sd3[i][0] + average_values.get("Support Devices", 0)) / 4
                output_lo[i][0] = (output_lo[i][0] + output_lo2[i][0] + output_lo3[i][0]) / 3
                max_output = max(output_pe[i][0], output_cm[i][0], output_fr[i][0], output_ec[i][0], output_lo[i][0])
                output_nf[i][0] = ((1 - max_output) + output_nf[i][0] + output_nf2[i][0]) / 3

                pe_score = output_pe[i][0] * 2 - 1
                cm_score = output_cm[i][0] * 2 - 1
                fr_score = output_fr[i][0] * 2 - 1
                ec_score = output_ec[i][0] * 2 - 1
                nf_score = output_nf[i][0] * 2 - 1
                lo_score = output_lo[i][0] * 2 - 1
                sd_score = output_sd[i][0] * 2 - 1
                pn_score = output_pn[i][0] * 2 - 1

                row = [batch_filenames[i]]
                for col in columns[1:]:
                    if col == "Pleural Effusion":
                        row.append(pe_score)
                    elif col == "Cardiomegaly":
                        row.append(cm_score)
                    elif col == "Fracture":
                        row.append(fr_score)
                    elif col == "Enlarged Cardiomediastinum":
                        row.append(ec_score)
                    elif col == "No Finding":
                        row.append(nf_score)
                    elif col == "Lung Opacity":
                        row.append(lo_score)
                    elif col == "Support Devices":
                        row.append(sd_score)
                    elif col == "Pneumonia":
                        row.append(pn_score)
                    else:
                        row.append(average_values.get(col, 0))

                predictions.append(row)

            batch = []
            batch_filenames = []

# Save all predictions
df_predictions = pd.DataFrame(predictions, columns=columns)
df_predictions = df_predictions.sort_values(by="Id")
df_predictions.to_csv('test_predictions.csv', index=False)

print("✅ Predictions saved to 'test_predictions.csv'")

Processing frontal:   0%|          | 0/19371 [00:00<?, ?it/s]

Processing lateral: 100%|██████████| 3289/3289 [02:22<00:00, 23.11it/s]


✅ Predictions saved to 'test_predictions.csv'


In [11]:
print(df_predictions['No Finding'].mean())

-0.76974934


In [12]:
df_predictions = pd.DataFrame(predictions, columns=columns)
df_predictions = df_predictions.sort_values(by="Id", ascending=True)
df_predictions.head()

,Id,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Pneumonia,Pleural Effusion,Pleural Other,Fracture,Support Devices
811,100018,-0.868912,-0.466074,0.483787,0.729600,0.988098,-0.799413,0.521795,0.570198,0.938017
12203,100019,-0.699381,-0.610575,-0.414551,0.948691,0.973979,0.003965,0.521795,0.182913,0.931121
14779,100022,-0.985295,0.043120,-0.031094,0.962553,0.997444,0.769150,0.521795,0.443930,0.942788
12851,100023,-0.980706,-0.008446,-0.132478,0.995996,0.999025,0.932050,0.521795,0.248880,0.942570
6047,100053,-0.568076,0.154119,0.737112,0.947145,0.996760,-0.617481,0.521795,0.439989,0.942054


In [ ]:
import os
print(os.getcwd())
